# Explore STEWMAP

Multiple datasets have been provided for this project.  I am working with NYC2017_STEWMAP_Version2.gdb.

This notebook will go through my standard analysis with a new dataset.  Specifically:

  1. Read the data into a geodataframe
  2. Check out the crs
  3. Look at the attributes
  4. Visualize
  5. Attribute analysis - query, subset, ... for further processing
  6. Office locations
  7. Save STEWMAP parquet files
  
The "output" of this analysis is an understanding of the STEWMAP dataset that I can use to fuse with other NYC open datasets.
  
**Note-to-self:** Document/explain the env, startup, imports, ...

# Read Data

The dataset comes as a geodatabase.  First I want to see what layers are included.  Once I know that, I'll read a layer to create a geopandas geodataframe.

In [ ]:
import fiona

In [ ]:
fiona.listlayers('../data/raw/NYC-2017-STEW-MAP-Public-Version2/NYC2017_STEWMAP_Version2.gdb/')

In [ ]:
%%time
office_locations_gdf = gpd.read_file('../data/raw/NYC-2017-STEW-MAP-Public-Version2/NYC2017_STEWMAP_Version2.gdb/',
                           driver='FileGDB',
                           layer=_[1])

In [ ]:
%%time
turfs_gdf = gpd.read_file('../data/raw/NYC-2017-STEW-MAP-Public-Version2/NYC2017_STEWMAP_Version2.gdb/',
                           driver='FileGDB',
                           layer='NYC2017_STEWMAP_PublicPolygons_Version2')

In [ ]:
print(f"Number of office locations: {len(office_locations_gdf)}")
print(f"Number of turfs: {len(turfs_gdf)}")

After office hour discussions, I am going to focus on the polygons (turfs).  Attributes are the same for each so ...  
I have no idea why there are more office locations then turfs.  The answer to that question is for another day!

# CRS

Since the ultimate objective is integration with other spatial datasets we need to understand the projection.  It is important for any spatial operations, i.e. joining, area/distance computations, ...

In [ ]:
turfs_gdf.crs

Simple idiom to embed an iframe with external resource.

Here's a description of the crs.  Note that the units for this projection are meters.

In [ ]:
IFrame("http://epsg.io/26918", width=1200, height=800)

# Dataframe Info

This section will look at the attributes.  

At first look, I wonder why all those attributes are encoded as float64. Is it an esri thing? I don't usually worry about sizes but ...

**Note:** Only looking at turfs for now.

**Note-to-self:** Can you make this more efficient?

In [ ]:
turfs_gdf.info(verbose=True, show_counts=True)

Lot's-o-variables here.  I wasn't sure how to deal with them at first.  During office hours Lindsay suggested I look at the [survey](https://www.nrs.fs.fed.us/STEW-MAP/resources/downloads/STEW-MAP%20NYC%202017%20Survey.pdf).  That definitely helped my understanding!

Check out the dtypes.  I see two attributes with int64 dtype.  The rest are either strings (object) or floats.  Will probably make sense to revisit and remap these dtypes.

**Note:** A slight digression, but something I realized further in the process.  According to the data dictionary, PopID is the unique ID.  Is it?

In [ ]:
turfs_gdf.PopID.is_unique

Hummm...

# Visualize

Finally, I like to see what's in the dataset.  This is not meant to be detailed mapping, just to get situated.  Let's look at two visualizations:

  1. the .plot function (matplotlib based)
  2. .explore method that is folium based

**Note-to-self:** I've created a subset but that doesn't seem to help.  Is it something about the geometries that slows it down?  Worth understanding.

In [ ]:
explore_gdf = turfs_gdf[['OrgName', 'OrgWebSite', 'OrgTwitter', 'geometry']].reset_index().drop(columns=['index'])

Note I'm reprojecting so the axis labels make some sense (lat/long).

In [ ]:
explore_gdf.to_crs("epsg:4326").plot(figsize=(18, 14));

Now we can look at the polygons with a map background.

In [ ]:
%%time
explore_gdf.explore(
    column='OrgName', 
    tooltip=['OrgName', 'OrgWebSite', 'OrgTwitter'],
    popup=True, 
    legend=False,
    width=1000,
    tiles="CartoDB positron", 
    cmap="Set1", 
    style_kwds=dict(color="black") 
    )

# Attribute Analysis

This last section will explore the attributes.  Remember the variables are best understood via the [survey](https://www.nrs.fs.fed.us/STEW-MAP/resources/downloads/STEW-MAP%20NYC%202017%20Survey.pdf).  Ultimately I need to develop "tools" to help fuse with other nyc datasets.

The first attribute is the `primary site type`, the one site type you work at the most.

I'll use the value_counts method to simply get the count of unique values.  You can plot as bar chart, but I just look at the numbers.  You can also normalize the count if you want percentages of total unique values.

In [ ]:
turfs_gdf.PrimST.value_counts()

Now we can look at `Primary Focus`.

In [ ]:
turfs_gdf.PrimFocus.value_counts()

One observation on the data are the fields with no value but they have a count.  Of course this type of thing happens in the real world.  You need to decide how to deal with it (infer, delete, exclude, ...)

Being curious I tried something -

In [ ]:
len(turfs_gdf.query(f"PrimST == ' '"))

In [ ]:
len(turfs_gdf.query(f"PrimFocus == ' '"))

So it seems like the `missing` value is just a blank character.  This might be important later, we'll defer for now.

Next I'll look at the organizations by state, zip, and city.

Remember I'm trying to understand how to build subsets of the data!

In [ ]:
turfs_gdf.OrgState.value_counts()

In [ ]:
turfs_gdf.OrgZip.value_counts()

In [ ]:
turfs_gdf.OrgCity.value_counts()[:20]

Now I'd like to start drilling into the data a bit.

For example the most selected PrimFocus was Environment.

In [ ]:
environment_turfs_gdf = turfs_gdf.query(f"PrimFocus == 'Environment'").reset_index().drop(columns=['index'])

What is the distribution of `primary site types` for this focus area?

In [ ]:
environment_turfs_gdf.PrimST.value_counts()

In [ ]:
environment_turfs_gdf[['OrgName', 'OrgWebSite', 'OrgTwitter', 'geometry']].explore(
    column='OrgName', 
    tooltip=['OrgName', 'OrgWebSite', 'OrgTwitter'],
    popup=True, 
    legend=False,
    width=1000,
    tiles="CartoDB positron", 
    cmap="Set1", 
    style_kwds=dict(color="black"))

In [ ]:
community_garden_turfs_gdf = turfs_gdf.query(f"PrimST == 'Community Garden'").reset_index().drop(columns=['index'])

In [ ]:
community_garden_turfs_gdf.PrimFocus.value_counts()

Another way to look at the turfs is by city (OrgCity).

In [ ]:
staten_island_gdf = turfs_gdf.query(f"OrgCity == 'Staten Island'").reset_index().drop(columns=['index'])

In [ ]:
staten_island_gdf[['OrgName', 'OrgWebSite', 'OrgTwitter', 'geometry']].explore(
    column='OrgName', 
    tooltip=['OrgName', 'OrgWebSite', 'OrgTwitter'],
    popup=True, 
    legend=False,
    width=1000,
    tiles="CartoDB positron", 
    cmap="Set1", 
    style_kwds=dict(color="black"))

Next we'll look at our first example `combining` STEWMAP with other data sources.

In [ ]:
waterfront_turfs_gdf = turfs_gdf.query(f"PrimST == 'Waterfront / Beach / Shoreline'").reset_index().drop(columns=['index'])

In [ ]:
waterfront_turfs_gdf[['OrgName', 'OrgWebSite', 'OrgTwitter', 'geometry']].explore(
    column='OrgName', 
    tooltip=['OrgName', 'OrgWebSite', 'OrgTwitter'],
    popup=True, 
    legend=False,
    width=1000,
    tiles="CartoDB positron", 
    cmap="Set1", 
    style_kwds=dict(color="black"))

In [ ]:
IFrame("https://www1.nyc.gov/site/planning/data-maps/open-data/dwn-wrp.page", width=1200, height=800)

In [ ]:
#fiona.listlayers('../data/NYCFutureHighTideWithSLR.gdb/')

In [ ]:
coastal_zone_gdf = gpd.read_file('../data/raw/NYCWRP_Shapefiles_2016/nycwrpsnwa_201601.shp')

In [ ]:
coastal_zone_gdf.explore()

In [ ]:
count_output = Output(layout={'border': '1px solid black',
                            'width': '50%'})

density_output = Output(layout={'border': '1px solid black',
                            'width': '50%'})

with count_output:
    display(staten_island_gdf[['OrgName', 'OrgWebSite', 'OrgTwitter', 'geometry']].explore())

with density_output:
    display(coastal_zone_gdf.explore())

#print('\nMaps for request type: ' + request_type + '\n\n')
HBox([count_output, density_output])

# Office locations revisited

To wrap up the analysis I want to examine the office locations.

A straight mapping of the geometry (in my tools of choice) did not work.  Need to investigate.

In [ ]:
office_locations_gdf.columns

In [ ]:
office_locations_gdf.iloc[27]

Notice the geometry is POINT Z, i.e. it contains elevation.  They all seem to be 0 so it doesn't look like it is used.  I am going to build the xy version for further analysis (possibly).

In [ ]:
new_geometry = [Point(x, y) for x, y in zip(office_locations_gdf.POINT_X, office_locations_gdf.POINT_Y)]

In [ ]:
new_geometry[0].wkt

In [ ]:
office_locations_gdf['geometry'] = new_geometry

In [ ]:
office_locations_gdf.explore()

# Save STEWMAP parquet

I am going to save the data as parquet files for upstream analysis.  I really should deal with the PopID here, but will wait until later.

In [ ]:
office_locations_gdf.to_parquet('../data/processed/office-locations.parq')

In [ ]:
turfs_gdf.to_parquet('../data/processed/turfs.parq')

